In [1]:
import pandas as pd
import glob

## Extract

In [2]:
df = pd.concat(map(pd.read_csv, glob.glob('test_input' + '\*.csv')))
#Extract All source code
#https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe

In [3]:
df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,jacareacanga,0,BR,1528902000,62,-6.22,-57.76,89.60,6.93
1,1,kaitangata,100,NZ,1528905304,94,-46.28,169.85,42.61,5.64
2,2,goulburn,20,AU,1528905078,91,-34.75,149.72,44.32,10.11
3,3,lata,76,IN,1528905305,89,30.78,78.62,59.89,0.94
4,4,chokurdakh,0,RU,1528905306,88,70.62,147.90,32.17,2.95


In [4]:
len(df)

1095

## Transform

In [5]:
df['hash'] = pd.Series((hash(tuple(row)) for _, row in df.iterrows()))
#Hash Code
#https://stackoverflow.com/questions/25757042/create-hash-value-for-each-row-of-data-with-selected-columns-in-dataframe-in-pyt
df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,hash
0,0,jacareacanga,0,BR,1528902000,62,-6.22,-57.76,89.60,6.93,-5869130011512737210
1,1,kaitangata,100,NZ,1528905304,94,-46.28,169.85,42.61,5.64,4031718688983238032
2,2,goulburn,20,AU,1528905078,91,-34.75,149.72,44.32,10.11,5021336569365925692
3,3,lata,76,IN,1528905305,89,30.78,78.62,59.89,0.94,1111483303566836649
4,4,chokurdakh,0,RU,1528905306,88,70.62,147.90,32.17,2.95,7518760468629069472


In [6]:
len(df)

1095

In [7]:
df_nodupe = df[~df.duplicated()]
#Dedupe code source
#https://stackoverflow.com/questions/40438237/assign-hash-to-row-of-categorical-data-in-pandas

In [8]:
df_nodupe.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,hash
0,0,jacareacanga,0,BR,1528902000,62,-6.22,-57.76,89.60,6.93,-5869130011512737210
1,1,kaitangata,100,NZ,1528905304,94,-46.28,169.85,42.61,5.64,4031718688983238032
2,2,goulburn,20,AU,1528905078,91,-34.75,149.72,44.32,10.11,5021336569365925692
3,3,lata,76,IN,1528905305,89,30.78,78.62,59.89,0.94,1111483303566836649
4,4,chokurdakh,0,RU,1528905306,88,70.62,147.90,32.17,2.95,7518760468629069472


In [9]:
len(df_nodupe)

548

## Load

#### > Convert DataFrame to json formatted file

In [10]:
import json

In [11]:
df_nodupe.to_json(path_or_buf=None, orient='index')
# Converting dataframe data to json format
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_json.html

'{"0":{"City_ID":0,"City":"jacareacanga","Cloudiness":0,"Country":"BR","Date":1528902000,"Humidity":62,"Lat":-6.22,"Lng":-57.76,"Max Temp":89.6,"Wind Speed":6.93,"hash":-5869130011512737210},"1":{"City_ID":1,"City":"kaitangata","Cloudiness":100,"Country":"NZ","Date":1528905304,"Humidity":94,"Lat":-46.28,"Lng":169.85,"Max Temp":42.61,"Wind Speed":5.64,"hash":4031718688983238032},"2":{"City_ID":2,"City":"goulburn","Cloudiness":20,"Country":"AU","Date":1528905078,"Humidity":91,"Lat":-34.75,"Lng":149.72,"Max Temp":44.32,"Wind Speed":10.11,"hash":5021336569365925692},"3":{"City_ID":3,"City":"lata","Cloudiness":76,"Country":"IN","Date":1528905305,"Humidity":89,"Lat":30.78,"Lng":78.62,"Max Temp":59.89,"Wind Speed":0.94,"hash":1111483303566836649},"4":{"City_ID":4,"City":"chokurdakh","Cloudiness":0,"Country":"RU","Date":1528905306,"Humidity":88,"Lat":70.62,"Lng":147.9,"Max Temp":32.17,"Wind Speed":2.95,"hash":7518760468629069472},"5":{"City_ID":5,"City":"martyush","Cloudiness":92,"Country":"RU

In [12]:
json_data = df_nodupe.to_json(orient='index')
parsed = json.loads(json_data)
# print(json.dumps(parsed, indent=4, sort_keys=True))
# Un-comment print to pretty print json formated data

#### > Create MongoDB named 'etl_db' with a collection 'etl_data'

In [13]:
import pymongo
from pymongo import MongoClient
client = MongoClient()
client = MongoClient('localhost', 27017)
client = MongoClient('mongodb://localhost:27017/')
print('Mongo version', pymongo.__version__)

Mongo version 3.9.0


In [14]:
# Uncomment code to drop existing database before creating new one
# client.drop_database('etl_db')

In [15]:
db = client['etl_db']
# Create database named 'etl_db'

db.etl_data.drop() # Drop collection if it already exist

collection = db['etl_data']
# Create collection called 'etl_data'

#### > Load ETL_json_file data

In [16]:
with open('ETL_json_file') as f:
    file_data = json.load(f)
    collection.insert_one(file_data) 
client.close()
# Read json file into mongoDB
# https://stackoverflow.com/questions/49510049/how-to-import-json-file-to-mongodb-using-python

In [17]:
db.list_collection_names()
# List collections in database

['etl_data']

In [18]:
db.etl_data.find( { 0: {} } )


In [19]:
# cursor = db.etl_data # selecting the etl_data collection
# for document in cursor.find():
#    print (document)
# Printing the content of the collection LARGE PRINTOUT

#### > Reading content of database and loading it into a dataframe

In [20]:
extracted_etl_data = db.etl_data
df = pd.DataFrame(list(extracted_etl_data.find()))
# https://stackoverflow.com/questions/16249736/how-to-import-data-from-mongodb-to-pandas

In [21]:
df

,0,1,10,100,101,102,103,104,105,106,...,91,92,93,94,95,96,97,98,99,_id
0,"{'City_ID': 0, 'City': 'jacareacanga', 'Cloudi...","{'City_ID': 1, 'City': 'kaitangata', 'Cloudine...","{'City_ID': 10, 'City': 'puerto ayora', 'Cloud...","{'City_ID': 100, 'City': 'auchel', 'Cloudiness...","{'City_ID': 101, 'City': 'bubaque', 'Cloudines...","{'City_ID': 102, 'City': 'khatanga', 'Cloudine...","{'City_ID': 103, 'City': 'los banos', 'Cloudin...","{'City_ID': 104, 'City': 'labuhan', 'Cloudines...","{'City_ID': 105, 'City': 'saint-paul', 'Cloudi...","{'City_ID': 106, 'City': 'petatlan', 'Cloudine...",...,"{'City_ID': 91, 'City': 'mataura', 'Cloudiness...","{'City_ID': 92, 'City': 'buala', 'Cloudiness':...","{'City_ID': 93, 'City': 'eyl', 'Cloudiness': 2...","{'City_ID': 94, 'City': 'abu dhabi', 'Cloudine...","{'City_ID': 95, 'City': 'tahe', 'Cloudiness': ...","{'City_ID': 96, 'City': 'chuy', 'Cloudiness': ...","{'City_ID': 97, 'City': 'dzerzhinskoye', 'Clou...","{'City_ID': 98, 'City': 'vanavara', 'Cloudines...","{'City_ID': 99, 'City': 'muros', 'Cloudiness':...",5d71c1360a1d8c971157e421


In [22]:
df = df.melt()

In [23]:
df.head()

,variable,value
0,0,"{'City_ID': 0, 'City': 'jacareacanga', 'Cloudi..."
1,1,"{'City_ID': 1, 'City': 'kaitangata', 'Cloudine..."
2,10,"{'City_ID': 10, 'City': 'puerto ayora', 'Cloud..."
3,100,"{'City_ID': 100, 'City': 'auchel', 'Cloudiness..."
4,101,"{'City_ID': 101, 'City': 'bubaque', 'Cloudines..."


In [24]:
len(df)

549

In [25]:
df.iloc[0][1]
# Showing content of first row 'value'

{'City_ID': 0,
 'City': 'jacareacanga',
 'Cloudiness': 0,
 'Country': 'BR',
 'Date': 1528902000,
 'Humidity': 62,
 'Lat': -6.22,
 'Lng': -57.76,
 'Max Temp': 89.6,
 'Wind Speed': 6.93,
 'hash': -3010865062464850435}

#### > CSV Move script - moves .csv files after being ETL processed

In [26]:
import shutil, os, glob

In [27]:
#For Loop moving all .csv files from the test_input folder 
for filePath in glob.glob('test_input' + '\*.csv'):
            #Move each file to the destination directory
        shutil.move(filePath, 'test_processed');
# Move all files that have been ETL'ed into a processed folder
#Source Code           
#https://thispointer.com/python-how-to-move-files-and-directories/